## 2.3 Detect Object Orientation using YOLO OBB

#### 2.3.1 Intro To YOLO Model

- Yolo (You Only Look Once) is single stage object detection model that makes predictions of bounding boxes and class probabilities all at once with a realtime processing speed.
- Historical timeline Yolo Development : <br>
<img src="resource/yolo-timeline.jpg" width="800px"><br><br>
- `Yolo` Conceptual Design :
    - General design Yolo architecture :<br>
    <img src="resource/yolo-archi-general.png" width="700px">

#### 2.3.2 What is YOLOv8?
- <font color="orange">YOLOv8</font> is a new state-of-the-art computer vision model built by [Ultralytics](https://www.ultralytics.com/), the creators of <font color="orange">YOLOv5</font>. 
- The <font color="orange">YOLOv8</font> model contains out-of-the-box support for <font color="orange">object detection</font>, <font color="orange">classification</font>, and <font color="orange">segmentation tasks</font>, accessible through a Python package as well as a command line interface.<br>
<img src="resource/YOLOv8_YouTube_Thumbnail-p-500.png" width="400px">

- <font color="orange">Yolo Model</font> can be downloaded from <font color="orange">Ultralytics Assets</font> repository : https://github.com/ultralytics/assets/releases/
- All model released in [Pytorch](https://pytorch.org/) Model format (`.pt`)
- For example if we want to use Yolo v11, there is several naming convention like the following,
- Based on Model Size :
    - `yolov8n.pt` : *Nano*,
    - `yolov8s.pt` : *Small*,
    - `yolov8m.pt` : *Medium*,
    - `yolov8b.pt` : *Balanced*,
    - `yolov8l.pt` : *Large*,
    - `yolov8x.pt` : *Extra Large*,
- Additional model with special capability :
    - `yolov8l-cls.pt` : *Classification Model*
    - `yolov8l-human.pt` : *Human Detection Model*
    - `yolov8l-obb.pt` : <font color="orange">*Oriented Bounding Box Object Detection*</font>
    - `yolov8l-pose.pt` : *Pose Estimation Model*
    - `yolov8l-seg.pt` : *Segmentation Model*<br>
    <img src="resource/ultralytics.png" width="800px">

#### 2.3.3 Yolo OBB Model
<img src="resource/ships-detection-using-obb.png" width="50%">

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt

print("OpenCV version: " + cv2.__version__)

#### EXAMPLE 1 - Using Ultralytics Yolo OBB Model to detect Object Orientation

In [ ]:
# Load a model
model = YOLO("yolov8l-obb.pt")  # load an official model

In [ ]:
# load image
image_path = "plane.jpg"
image = cv2.imread(image_path)

# Predict with the model
results = model(image_path)  # predict on an image

In [ ]:
# Draw bounding boxes and orientations
output = image.copy()
for result in results[0]:

    xywhr = result.obb.xywhr.cpu().numpy().squeeze()  # Extract [center-x, center-y, width, height, angle] from the result
    
    center = tuple(xywhr[:2].astype(int))  # Extract center (x, y)
    
    # Extract rotation angle in radians, 
    # considering the object is taller than it is wide, 
    # otherwise, we need to add np.pi to the angle
    if(xywhr[2] < xywhr[3] ):
        angle_rad = xywhr[4] 
    else:
        angle_rad = np.pi - xywhr[4] # flip the angle
    
    # Define the length of the orientation lines
    length = 50 

    # Draw original horizontal reference line (black)
    cv2.line(output, center, (center[0] + length, center[1]), (0, 0, 0), 2)
    
    # Compute and draw rotation axis (red line)
    end_x = int(center[0] + length * np.cos(angle_rad))
    end_y = int(center[1] - length * np.sin(angle_rad))
    cv2.line(output, center, (end_x, end_y), (0, 0, 255), 2)
    
    # Draw angle text
    angle = np.rad2deg(angle_rad)
    text_position = (center[0] - 7, center[1] - 7)
    cv2.putText(output, f"{int(angle)}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, (0, 0, 0), 1, cv2.LINE_AA)

# Show results using Matplotlib
plt.figure(figsize=(20, 6))
plt.imshow(output[:, :, ::-1])  # Convert BGR to RGB
plt.axis("off")
plt.title("Object Orientation with Reference Line")
plt.show()


- **Pros ✅**
    - ✔ **Robust to Noise & Partial Occlusion** – Learns object features beyond shape.  
    - ✔ **Works with Any Object Type** – Can detect orientation even for irregular shapes.  
    - ✔ **Combines Object Detection & Orientation** – Provides classification along with orientation.  
    - ✔ **Performs Well in Real-World Scenarios** – Ideal for aerial images, robotics, and autonomous systems.  

- **Cons ❌**
    - ✖ **Requires Training Data & Annotations** – Needs a labeled dataset with object orientations.  
    - ✖ **Computationally Expensive** – Requires a GPU for real-time inference.  

# Source : 

- https://docs.ultralytics.com/tasks/obb/